# Preface

The main code of CGSB was created independently from insane, but the lipid, solvent and protein definitions used in insane have been converted to the new CGSB data format.

# General feature overview:


**Bold** statements are either additions or improvements compared to the python2 insane.py script

*Italic* statements are planned but not yet implemented

General:
- **Can be either imported and run from a python script/jupyter or run directly from the terminal**
    - **Commands are slightly different depending due to difference in terminal/python syntax**
- Can create "cubic", hexagonal and rhombic dodecahedron (with hexagonal xy-plane) system
- *insane.py lipid/solvent definitions converter*
- *insane.py command to CGSB command translator*

Membrane creation:
- All membranes are created in the xy-plane.
- Individual leaflets in membranes can be created independently of each other.
- **Any number of membranes can be created, though overlaps between different membranes is not checked for, so use with caution.**
- **The number of lipids in a leaflet is calculated exactly, based on the APL (area per lipid) of the leaflet, taking into account the leaflet surface area occupied by proteins.**
- **All calculations are done on a leaflet-by-leaflet basis**
    - **Multiple methods available for optimizing ratio between lipid types within a given leaflet**
- **Membranes are created by inserting an exact number of lipids in a leaflet after which the distance between the lipids is "minimized" using a custom written algorithm to prevent potential bead overlaps**
- **CGSB utilizes the powerful geometry module called "shapely" to handle abnormalities present in a membrane (such as protein-occupied areas (Working) and *artificially created holes (WIP)***
- **Dynamically splits large membranes into multiple smaller ones to enhance performance**

Protein insertion:
- Structure files (pdb/gro) can be inserted as "proteins"
- Any number of structures files can be inserted
- Structures can be moved and rotated based on the structure's center
    - Multiple methods available for designating the structure's center
    - *Allow for centering based on hydrophobic regions (OPM/PPM*
- *Allow for multi-insertions of the same protein*

Solvation:
- **The number of non-ionic solvent molecules is calculated based on the free volume of the box**
    - **The free volume is estimated using the number of other particles present in the box**
- **The number of ionic solvent molecules is calculated based on the solvent volume**
    - **Estimated using the number of non-ionic solvent particles created using the same command**
- Solvent placement algorithm ensures no solvent is placed within the hydrophobic volume of membranes and ensures a minimum distance between solvent and other particles
- **Any number of solvations can be given**
- **Structures (pdb/gro) can be imported and used as solvent molecules**

Charge calculations:
- Topology files can be given from which charge information will be read
- Understands definitions and recursively runs "#include Path/To/top.itp" statements
    - For definitions and #include statements to work correctly they must be given in the same order as one would use for "gmx grompp"
- Lipids and solvent will their names directly correlated with names in the topology
- Protein files must have their name(s) specified in their command




# CGSB workflow - OUTDATED

A point is only run if its prerequisites are fulfilled
1. Ensure commands given are understood
1. Process imported topology/itp file(s)
1. Import structures to structure libraries
1. Preprocess lipid, solvent and ion definitions
1. Preprocess protein, leaflet and solvent commands

1. Place proteins
1. Create Bounding Box for membrane leaflets
1. Create holed leaflets depending on proteins
1. Calculated number of lipids
1. Create lipid grid
1. Minimize lipid grid
1. Insert lipids
1. Solvate system

1. pdb, gro, top and log files are written


# Command and subcommand syntax

CGSB functions on a system of commands and subcommands as shown here:\
All "command_call" are simply stand-ins for real commands
- Run from script:          CGSB(    command_call = "cmd")
- Run from terminal: python CGSB.py -command_call cmd

Multiple calls to the same command
- Run from script:   command_call = [”cmd1”,  ”cmd2”]
- Run from script (alternative):   command_call1 = ”cmd1”, command_call2 = ”cmd2”
- Run from terminal: -command_call cmd1 -command_call cmd2


Subsequent subcommands are given as SETTING:VAL1:VAL2 pairs where the string prior to the first colon designates the setting to be changed, while following values separated by colons are the values to be given to the setting.

The number of values a setting can be given depends on the specific setting.

Subcommands given to a command call only affects that specific command


# Importing and command explanations - SLIGHTLY OUTDATED

This section will explain the various commands and showcase some of their more useful subcommands

## Packages

The following packages are needed for all features of CGSB to run:
- ast
- itertools
- re
- math
- numpy as np
- random
- copy
- scipy.spatial
- alphashape
- shapely
- shapely.plotting
- inspect
- sys
- os
- argparse
- operator
- time
- matplotlib
- pickle

## General system commands

The "-box X Y Z" command is used to set the sidelengths of the box in x/y/z. The commands "x"/"y"/"z" can instead be used to set the coordinates individually
- box = [10, 10, 10]
- x = 10, y = 10, z = 10

The "-params" command can be used to set the "standard" parameters used in "membrane" and "solvation" commands. This can be used to have multiple versions of the lipid. "params" can also be set individually per "membrane"/"solvation" command or for specific lipids/solvents, but that will be shown further down.
- params = "default" (default)
- params = "dev18"
- params = "PhosV13"

The "-sn" command can be used to set the name of the outputted system.
- sn = "Tutorial System"

The "-rand" command can be used to set the random seed.
- rand = 5

The "itp_input" command can be used to give the program topology data which will be used to calculate charges.
- itp_input = "top_for_CGSB.itp"

"-h" will cause the program to print the help.


## Protein insertions ("protein" / "prot")

The "protein" command is used to insert structures into specific positions

The first subcommand must always be a path to the file that is to be imported
- protein = "Protein.pdb"
- prot = "Protein.pdb"

The structure can be moved using tx/ty/tz (in nm) and rotated using rx/ry/rz (in degrees)
- protein = "Protein.pdb **tx:5 tz:3 ry:90**"

The centering subcommand can be changed using "cen_method" and has the following uses:
- "cen_method:**cog**" (default) - Centers on the mean coordinate of all beads (center of geomtry)
- "cen_method:**axis**" - Centers on the axial mean coordinate
- "cen_method:**bead:beadnr**" - Centers on a specific bead number
- "cen_method:**res:resnr**" - Centers on the mean coordinate of a specific residue number
- "cen_method:**point:x:y:z**" - Centers on a specific point. Uses the coordinate system of the imported file.

If one wants to use itp/topology files to obtain the charge of a structure then the name(s) to be found can be designated used "mol_names". If protein names cannot be found in the topology then the program will revert to looking up amino acid charges.
- protein = "Protein.pdb mol_names:PROTEIN1:LIGAND1:LIGAND1"
- This will ensure that the charge from one instance of PROTEIN1 and two instances of LIGAND1 are included



## Membrane creation ("membrane" / "memb")

The "membrane" command is used to create membranes.

Lipids can be added to a "leaf" command by simply adding their name to the command along with a number indicating the ratios between lipids in the leaflet. 
- membrane = "POPC:5"
- memb = "POPC:5"

Area per lipid (apl) can be used to control how tightly a membrane/leaflet is packed.
- "apl:0.6" (default)
- "apl:0.7" --> more sparsely packed
- "apl:0.5" --> more tightly packed

If another string is added after the number, then the lipid will be looked for in that specific set of parameters. The "params" subcommand can be used to set the default parameters for the specific "leaf" command. Lipid-specific ff designations overwrites the leaflet ff designation if given:
- membrane = "POPC:5 POPE:2.5 CHOL:1:dev18 params:PhosDev13" - POPC and POPE would use "PhosDev13" and CHOL would use "dev18"

Whether a "membrane" command should create a "bilayer", or standalone "upper" or "lower" leaflets can be designated using the "type" subcommand:
- "type:bilayer" (default) --> symmetric bilayer
- "type:mono" --> upwards pointing monolayer
- "type:upper" --> upwards pointing monolayer
- "type:lower" --> downwards pointing monolayer

Asymmetry can be created using either multiple membrane commands:
- membrane = ["type:upper POPC:5 POPE:2 CHOL:1", "type:lower POPC:3 POPE:3 CHOL:2"]

Or by using the "leaflet" subcommand, wherein subcommands will only be applicable to the last called "leaflet", with "leaflet:membrane" designating that following commands should affect the entire membrane.
- membrane = "leaflet:upper POPC:5 POPE:2 CHOL:1 leaflet:lower POPC:3 POPE:3 CHOL:2 leaflet:membrane apl:0.5"

Leaflets will by default fill the entire xy-plane but their sizes can be changed using x/y along with their center. This can be used to create patches with different starting lipid compositions. Note that the system is centrosymmetric during calculations so center:X:Y:Z commands should be given based on that and be in ${nm}$. It should also be noted that each leaflet is treated independently in calculations and thus APL calculations might be slightly off due to multiple roundings.
- box = [10,10,10], membrane = ["POPC:5 CHOL:1 x:5 center:2.5:0:0", "POPC:4 CHOL:2 x:5 center:-2.5:0:0"]

The method used for converting lipid ratios to actual lipids can be chosen using "lipid_optim"
- "lipid_optim:avg_optimal" (default) - Lipids are optimized such that the mean lipid deviation from the expected ratios is used.
- "lipid_optim:abs_val" - Treats lipid ratios as actual number of lipids
- "lipid_optim:fill" - Attempts to fill the leaflet regardsless of how skewed the ratios would become. Stops if perfect ratio reached
- "lipid_optim:force_fill" - Same as "fill" but forces the leaflet to be filled completely
- "lipid_optim:no" - Does not attempt to optimize the lipid ratios (Works identically to insane.py)





## Solvations ("solvation" / "solv")

The "solvation" command is used to solvate the system.

Non-ionic solvent can be added using "solv" while positive and negative ionic solvent can be added using "pos" and "neg" respectively:
- solvation = "solv:W pos:NA neg:CL"

If one supplies an empty string to the command then it will automatically be treated as "solv:W pos:NA neg:CL"
- solvation = "" --> "solv:W pos:NA neg:CL"

The "params" subcommand can be used to set the default parameters for the specific "solvent" command. solvent-specific parameter designations can also be made similarly to how it is done for lipids:
- solvation = "solv:W:DevWater5 pos:NA neg:CL params:DevIons6"

The molarity (atomistic molarity) of the solvent and ions can be set using "solv_molarity" and "salt_molarity" respectively:
- solvation = "solv:W pos:NA neg:CL solv_molarity:55.56 salt_molarity:0.15" (default values shown)

Different solvents and ions can be added in different ratios designated by a number after the solvent name:
- solvation = "solv:W:5 solv:SW:2 pos:NA:5 pos:CA:1 neg:CL"

Solvents from different params can be added in different ratios as such, where the params is written after the ratio:
- solvation = "solv:W:5:DevWater4 solv:SW:2:DevWater5 pos:NA neg:CL"

Multiple solvations can be made to for example insert X number of ligands. Note that the solvation command that fills the box with water and ions should go last. The "count" command makes it such that the molarity is consideres as the actual number of solvent molecules.
- solvation = ["solv:TRP count:True solv_molarity:20", "solv:W pos:NA neg:CL"]


## Import solute for solvations ("solute_input" / "-solute_in")

The "solute_input" command can be used to import structures, which are added to the solvent/ion libraries.

Solute_input commands can be given in two ways:
Version 1:
- Single subcommand written as mol:name:n_residues:charge
    - each "mol" call is new molecule
    - name can be either "resname" or a string of up to 4 characters
    - n_residues must be integer value
    - charge must be either int/float value or string of topology moleculetype
    - if no charge given, assume top using name
    - if no n_residues given along with no charge then assume 1 residue
- Default:
    - If no "mol" subcommands given. Assume all are single-residue molecules
    - Residue name used for "name" and "moleculetype"
    - If no moleculetype found for "name", then assume charge of 0
    - Examples: "ligands.pdb   mol:RHO:1:rhodamine   mol:PEP1:8:peptide_1   mol:resname:1:2"
   
Version 2:
- multiple subcommands:
    - names: list of names
    - n_residues/nres: list of number of residues
    - charges: list of charge designations
    - Same number of values must be given to each of above subcommands
- Default:
    - names: required unless n_residues 1 for all residues, for which resname will be used
    - n_residues/nres: assumed 1 residue for all molecules
    - charges: assumed "top" using names. Defaults to charge of 0 if no top found

Examples:
- V1: solute_input = "LIGANDS.pdb mol:LIG1:2:2 mol:LIG2:3:0"
- V2: solute_input = "LIGANDS.pdb names:LIG1:LIG2 nres:2:3 charge:2:0"

- solvation = ["solv:LIG1 count:True solv_molarity:20 params:ligands", "solv:W pos:NA neg:CL"]


# Example commands

## Basic System

In [8]:
import CGSB
''' Basic system

Contains a symmetric membrane composed entirely of POPC

The system is solvated using default molarities for water, potassium ions and chloride ions
'''

sysname = "BasicSystem"
test_outdir = "/".join(["output_test_systems", sysname])
!mkdir -p {test_outdir}

### From script:
CGSB.CGSB(
    pbc = [10, 10, 10],
    
    membrane = "POPC:1",
    
    solvation = "", # if no command is given to solvation, then it defaults to "solv:W pos:NA neg:CL"
    
    out_sys = test_outdir + "/" + "_".join(["output", sysname]),
    out_top = test_outdir + "/" + "_".join(["topol", sysname]) + ".top",
    out_log = test_outdir + "/" + "_".join(["log", sysname]) + ".log",
    
    backup = False,
)


------------------------------ PREPROCESSING
Preprocessing lipid definitions
     Number of lipids preprocessed: 107 

Preprocessing solvent definitions
     Number of solvents preprocessed: 22 

Preprocessing ion definitions
     Number of positive ions preprocessed: 3
     Number of negative ions preprocessed: 9

Preprocessing leaflet requests
     Starting leaflet command: 1
     Number of membranes preprocessed: 1

Preprocessing solvent requests
     Starting Solvent command: 1
     Number of solvent commands preprocessed: 1
------------------------------ PREPROCESSING COMPLETE 

------------------------------ CREATING LEAFLET BOUNDARY BOXES
Making boundary boxes for membrane nr 1
     Starting upper leaflet
          Base parameters: x=10.0nm y=10.0nm APL=0.6nm^2

     Starting lower leaflet
          Base parameters: x=10.0nm y=10.0nm APL=0.6nm^2
------------------------------ LEAFLET BOUNDARY BOXES CREATED 

------------------------------ CREATING HOLED BOUNDARY BOXES
Starting m

## Simple System

In [10]:
import CGSB
''' Simple system

Contains a symmetric membrane composed of POPC and CHOL in a 5:1 ratio

The system is solvated using default molarities for water, potassium ions and chloride ions

Charge data is gathered from the "top_for_CGSB.top"
'''

sysname = "SimpleSystem"
test_outdir = "/".join(["output_test_systems", sysname])
!mkdir -p {test_outdir}

### From script:
CGSB.CGSB(
    pbc = [10, 10, 10],
    
    membrane = "POPC:5 CHOL:1",
    
    solvation = "",
    
    itp_input = "top_for_CGSB.top",
    
    out_sys = test_outdir + "/" + "_".join(["output", sysname]),
    out_top = test_outdir + "/" + "_".join(["topol", sysname]) + ".top",
    out_log = test_outdir + "/" + "_".join(["log", sysname]) + ".log",
    
    backup = False,
)



Loading topology file(s)
     Finished loading topologies. Number of moleculetypes found: 351
------------------------------ PREPROCESSING
Preprocessing lipid definitions
     Number of lipids preprocessed: 107 

Preprocessing solvent definitions
     Number of solvents preprocessed: 22 

Preprocessing ion definitions
     Number of positive ions preprocessed: 3
     Number of negative ions preprocessed: 9

Preprocessing leaflet requests
     Starting leaflet command: 1
     Number of membranes preprocessed: 1

Preprocessing solvent requests
     Starting Solvent command: 1
     Number of solvent commands preprocessed: 1
------------------------------ PREPROCESSING COMPLETE 

------------------------------ CREATING LEAFLET BOUNDARY BOXES
Making boundary boxes for membrane nr 1
     Starting upper leaflet
          Base parameters: x=10.0nm y=10.0nm APL=0.6nm^2

     Starting lower leaflet
          Base parameters: x=10.0nm y=10.0nm APL=0.6nm^2
------------------------------ LEAFLET B

## System With Protein

In [13]:
import CGSB
''' System With Protein

Contains a symmetric membrane composed of POPC and CHOL in a 4.7:1.3 ratio with an APL of 0.536
causing more lipids to be placed in the membrane than with the default APL of 0.6

A protein is inserted at the center of the system with its topology name being "PROT_1"
### Change protein.pdb to be your protein
### Chain PROT_1 to be the mol_name in you .itp file

The system is solvated using default molarities for water, potassium ions and chloride ions

Charge data is gathered from the "top_for_CGSB.top" file

Written files are given custom names and a log file is written
'''

sysname = "SystemWithProtein"
test_outdir = "/".join(["output_test_systems", sysname])
!mkdir -p {test_outdir}

### From script:
CGSB.CGSB(
    pbc = [10, 10, 10],
    
    protein = "protein.pdb mol_names:PROT_1",
    
    membrane = "apl:0.536 POPC:4.7 CHOL:1.3",
    
    solvation = "",
    
    itp_input = "top_for_CGSB.top",
    
    out_sys = test_outdir + "/" + "_".join(["output", sysname]),
    out_top = test_outdir + "/" + "_".join(["topol", sysname]) + ".top",
    out_log = test_outdir + "/" + "_".join(["log", sysname]) + ".log",
    
    backup = False,
)



Loading topology file(s)
     Finished loading topologies. Number of moleculetypes found: 351
------------------------------ PREPROCESSING
Preprocessing lipid definitions
     Number of lipids preprocessed: 107 

Preprocessing solvent definitions
     Number of solvents preprocessed: 22 

Preprocessing ion definitions
     Number of positive ions preprocessed: 3
     Number of negative ions preprocessed: 9

Preprocessing protein requests
     Starting protein: 1
     Number of molecule insertions preprocessed: 1

Preprocessing leaflet requests
     Starting leaflet command: 1
     Number of membranes preprocessed: 1

Preprocessing solvent requests
     Starting Solvent command: 1
     Number of solvent commands preprocessed: 1
------------------------------ PREPROCESSING COMPLETE 

------------------------------ PROTEIN PLACEMENT
Starting protein nr 1
     Centering protein using 'cog' at x/y/z: 0.0 -0.0 0.0 (Input file coordinate system [Å])
     New protein center at x/y/z: 0 0 0 (I

## Asymmetric Membrane System

In [15]:
import CGSB
''' Asymmetric Membrane System

Contains an asymmetric membrane
- The upper leaflet is composed of POPC and CHOL in a 4.7:1.3 ratio with an APL of 0.573 and
- The lower leaflet is composed of POPC and CHOL in a 3.8:2.8 ratio with an APL of 0.487 and
- The APLs will cause the lower leaflet to contain more lipids than the upper one

The system is solvated using default molarities for water, potassium ions and chloride ions

Charge data is gathered from the "top_for_CGSB.top" file
'''

sysname = "AsymmetricMembraneSystem"
test_outdir = "/".join(["output_test_systems", sysname])
!mkdir -p {test_outdir}

### From script:
CGSB.CGSB(
    pbc = [10, 10, 10],
    
    membrane = "leaflet:upper apl:0.573 POPC:4.7 CHOL:1.3 leaflet:lower apl:0.487 POPC:3.8 CHOL:2.8",
    
    solvation = "",
    
    itp_input = "top_for_CGSB.top",
    
    out_sys = test_outdir + "/" + "_".join(["output", sysname]),
    out_top = test_outdir + "/" + "_".join(["topol", sysname]) + ".top",
    out_log = test_outdir + "/" + "_".join(["log", sysname]) + ".log",
    
    backup = False,
)



Loading topology file(s)
     Finished loading topologies. Number of moleculetypes found: 351
------------------------------ PREPROCESSING
Preprocessing lipid definitions
     Number of lipids preprocessed: 107 

Preprocessing solvent definitions
     Number of solvents preprocessed: 22 

Preprocessing ion definitions
     Number of positive ions preprocessed: 3
     Number of negative ions preprocessed: 9

Preprocessing protein requests
     Starting protein: 1
     Number of molecule insertions preprocessed: 1

Preprocessing leaflet requests
     Starting leaflet command: 1
     Number of membranes preprocessed: 1

Preprocessing solvent requests
     Starting Solvent command: 1
     Number of solvent commands preprocessed: 1
------------------------------ PREPROCESSING COMPLETE 

------------------------------ PROTEIN PLACEMENT
Starting protein nr 1
     Centering protein using 'cog' at x/y/z: 0.0 -0.0 0.0 (Input file coordinate system [Å])
     New protein center at x/y/z: 0 0 0 (I

## Phase Separated System

In [1]:
import CGSB
''' Phase Separated System

Contains phase separated membrane constructed from 4 separate membrane commands

The system is solvated using default molarities for water, potassium ions and chloride ions

Charge data is gathered from the "top_for_CGSB.top" file
'''

sysname = "PhaseSeparatedSystem"
test_outdir = "/".join(["output_test_systems", sysname])
!mkdir -p {test_outdir}

### From script:
CGSB.CGSB(
    pbc = [20, 20, 10],
    
    membrane = [
        ### center:x:y:z
        "DPPC:5 CHOL:1 center:-5:-5:0 x:10 y:10", # bottom left
        "DOPC:5 CHOL:1 center:5:-5:0 x:10 y:10",  # bottom right
        "DOPC:5 CHOL:1 center:-5:5:0 x:10 y:10",  # top left
        "DPPC:5 CHOL:1 center:5:5:0 x:10 y:10",   # top right
        
        ### Results in 5:5:2 DPPC:DOPC:CHOL distribution
    ],
    
    solvation = "",
    
    itp_input = "top_for_CGSB.top",
    
    out_sys = test_outdir + "/" + "_".join(["output", sysname]),
    out_top = test_outdir + "/" + "_".join(["topol", sysname]) + ".top",
    out_log = test_outdir + "/" + "_".join(["log", sysname]) + ".log",
    
    backup = False,
)



Loading topology file(s)
     Finished loading topologies. Number of moleculetypes found: 351
------------------------------ PREPROCESSING
Preprocessing lipid definitions
     Number of lipids preprocessed: 107 

Preprocessing solvent definitions
     Number of solvents preprocessed: 22 

Preprocessing ion definitions
     Number of positive ions preprocessed: 3
     Number of negative ions preprocessed: 9

Preprocessing leaflet requests
     Starting leaflet command: 1
     Starting leaflet command: 2
     Starting leaflet command: 3
     Starting leaflet command: 4
     Number of membranes preprocessed: 4

Preprocessing solvent requests
     Starting Solvent command: 1
     Number of solvent commands preprocessed: 1
------------------------------ PREPROCESSING COMPLETE 

------------------------------ CREATING LEAFLET BOUNDARY BOXES
Making boundary boxes for membrane nr 1
     Starting upper leaflet
          Base parameters: x=10.0nm y=10.0nm APL=0.6nm^2

     Starting lower leafle

     Neighborlist: 0.0205
     LAST STEP:      3
     STEPS TIME:     0.04000711441040039
     MEAN STEP TIME: 4.000711441040039e-05
     MAXIMUM PUSH:   1.227566039651089e-05
Starting membrane nr 3
     Starting upper leaflet
     Binning:      0.0007
     Neighborlist: 0.0117
     LAST STEP:      3
     STEPS TIME:     0.03087902069091797
     MEAN STEP TIME: 3.087902069091797e-05
     MAXIMUM PUSH:   4.010969644015066e-05

     Starting lower leaflet
     Binning:      0.0007
     Neighborlist: 0.0114
     LAST STEP:      2
     STEPS TIME:     0.024936437606811523
     MEAN STEP TIME: 2.4936437606811522e-05
     MAXIMUM PUSH:   6.226638911869259e-05
Starting membrane nr 4
     Starting upper leaflet
     Binning:      0.0006
     Neighborlist: 0.0023
     LAST STEP:      3
     STEPS TIME:     0.01971602439880371
     MEAN STEP TIME: 1.971602439880371e-05
     MAXIMUM PUSH:   7.723720214275497e-05

     Starting lower leaflet
     Binning:      0.0006
     Neighborlist: 0.0021
    

## Phase Separated System Generator

In [4]:
import CGSB
import numpy as np
''' Phase Separated System Generator

Contains phase separated membrane constructed from a number of coloumns and rows

The system is solvated using default molarities for water, potassium ions and chloride ions

Charge data is gathered from the "top_for_CGSB.top" file
'''

sysname = "PhaseSeparatedSystemGenerator"
test_outdir = "/".join(["output_test_systems", sysname])
!mkdir -p {test_outdir}

### ### Initial settings:
pbc = [20, 20, 10]
xrows = 3
yrows = 5
### ###

xmin, xmax = -pbc[0]/2, pbc[0]/2
ymin, ymax = -pbc[1]/2, pbc[1]/2

xlen = round(pbc[0]/xrows, 3)
ylen = round(pbc[1]/yrows, 3)

xcenters = np.linspace(xmin+(xlen/2), xmax-(xlen/2), num=xrows, endpoint=True)
ycenters = np.linspace(ymin+(ylen/2), ymax-(ylen/2), num=yrows, endpoint=True)

centers = [(round(x,3), round(y,3)) for x in xcenters for y in ycenters]
lipids = []
memb_commands = []
DPPC_counter = 0
DOPC_counter = 0
for i, (x, y) in enumerate(centers):
    if i % 2 == 0:
        lipid = "DPPC"
        DPPC_counter += 1
    else:
        lipid = "DOPC"
        DOPC_counter += 1
    memb_commands.append(lipid+":5 center:"+str(x)+":"+str(y)+":0 x:"+str(xlen) + " y:"+str(ylen) )

print("DPPC patches:", DPPC_counter)
print("DOPC patches:", DOPC_counter)

## From script:
CGSB.CGSB(
    pbc = pbc,
    
    membrane = memb_commands,
    
    solvation = "",
    
    itp_input = "top_for_CGSB.top",
    
    out_sys = test_outdir + "/" + "_".join(["output", sysname]),
    out_top = test_outdir + "/" + "_".join(["topol", sysname]) + ".top",
    out_log = test_outdir + "/" + "_".join(["log", sysname]) + ".log",
    
    backup = False,
)


DPPC patches: 8
DOPC patches: 7

Loading topology file(s)
     Finished loading topologies. Number of moleculetypes found: 351
------------------------------ PREPROCESSING
Preprocessing lipid definitions
     Number of lipids preprocessed: 107 

Preprocessing solvent definitions
     Number of solvents preprocessed: 22 

Preprocessing ion definitions
     Number of positive ions preprocessed: 3
     Number of negative ions preprocessed: 9

Preprocessing leaflet requests
     Starting leaflet command: 1
     Starting leaflet command: 2
     Starting leaflet command: 3
     Starting leaflet command: 4
     Starting leaflet command: 5
     Starting leaflet command: 6
     Starting leaflet command: 7
     Starting leaflet command: 8
     Starting leaflet command: 9
     Starting leaflet command: 10
     Starting leaflet command: 11
     Starting leaflet command: 12
     Starting leaflet command: 13
     Starting leaflet command: 14
     Starting leaflet command: 15
     Number of membranes

STEP: 20
     Updating neigborlist
     Binning:      0.0003
     Neighborlist: 0.0041
     Max Push:     0.1082
STEP: 30
STEP: 40
     LAST STEP:      41
     STEPS TIME:     0.13005876541137695
     MEAN STEP TIME: 0.00013005876541137695
     MAXIMUM PUSH:   0.0996106938354911

     Starting lower leaflet
     Binning:      0.0003
     Neighborlist: 0.0044
STEP: 10
     Updating neigborlist
     Binning:      0.0003
     Neighborlist: 0.0039
     Max Push:     0.1705
STEP: 20
STEP: 30
STEP: 40
STEP: 50
     Updating neigborlist
     Binning:      0.0003
     Neighborlist: 0.0041
     Max Push:     0.1006
     LAST STEP:      51
     STEPS TIME:     0.15153074264526367
     MEAN STEP TIME: 0.00015153074264526368
     MAXIMUM PUSH:   0.09933783846862124
Starting membrane nr 7
     Starting upper leaflet
     Binning:      0.0002
     Neighborlist: 0.0034
     LAST STEP:      2
     STEPS TIME:     0.008501052856445312
     MEAN STEP TIME: 8.501052856445312e-06
     MAXIMUM PUSH:   0.09

## Mixed Solvent System

In [16]:
import CGSB
''' Mixed solvent system

Contains a symmetric membrane composed of POPC and CHOL in a 5:1 ratio

The system is solvated using default molarities for water, potassium ions and chloride ions
- The system contains a 5:3 ratio of regular water "W" to small water "SW"
- The system contains a 5:1 ratio of potassium ions to calcium ions
- The system's only negative ion is chloride

Charge data is gathered from the "top_for_CGSB.top" file
'''

sysname = "MixedSolventSystem"
test_outdir = "/".join(["output_test_systems", sysname])
!mkdir -p {test_outdir}

### From script:
CGSB.CGSB(
    pbc = [10, 10, 10],
    
    membrane = "POPC:5 CHOL:1",
    
    solvation = "solv:W:5 solv:SW:3 pos:NA:5 pos:CA:1 neg:CL",
    
    itp_input = "top_for_CGSB.top",
    
    out_sys = test_outdir + "/" + "_".join(["output", sysname]),
    out_top = test_outdir + "/" + "_".join(["topol", sysname]) + ".top",
    out_log = test_outdir + "/" + "_".join(["log", sysname]) + ".log",
    
    backup = False,
)



Loading topology file(s)
     Finished loading topologies. Number of moleculetypes found: 351
------------------------------ PREPROCESSING
Preprocessing lipid definitions
     Number of lipids preprocessed: 107 

Preprocessing solvent definitions
     Number of solvents preprocessed: 22 

Preprocessing ion definitions
     Number of positive ions preprocessed: 3
     Number of negative ions preprocessed: 9

Preprocessing leaflet requests
     Starting leaflet command: 1
     Number of membranes preprocessed: 1

Preprocessing solvent requests
     Starting Solvent command: 1
     Number of solvent commands preprocessed: 1
------------------------------ PREPROCESSING COMPLETE 

------------------------------ CREATING LEAFLET BOUNDARY BOXES
Making boundary boxes for membrane nr 1
     Starting upper leaflet
          Base parameters: x=10.0nm y=10.0nm APL=0.6nm^2

     Starting lower leaflet
          Base parameters: x=10.0nm y=10.0nm APL=0.6nm^2
------------------------------ LEAFLET B

## Flooding System

In [19]:
import CGSB
''' Flooding with imported ligand system

Contains a symmetric membrane composed of POPC and CHOL in a 5:1 ratio with the default APL of 0.6

A protein is inserted at the center of the system with its topology name being "PROT_1"
### Replace "Ligand_1.pdb" with your ligand file
### Replace LIG1_name with the name you want to refer to the ligand as
### Replace LIG1_charge with the charge of the ligand or the mol_name used in the topology file

The system is solvated first with 30 molecules of LIG1 after which it is solvated
using default molarities for water, potassium ions and chloride ions

Charge data is gathered from the "top_for_CGSB.top" file
'''

sysname = "FloodingSystem"
test_outdir = "/".join(["output_test_systems", sysname])
!mkdir -p {test_outdir}

### From script:
CGSB.CGSB(
    pbc = [10, 10, 10],
    
    membrane = "POPC:5 CHOL:1",
    
    solvation = [
        "solv:LIG1 count:True solv_molarity:30 params:imp",
        "solv:W pos:NA neg:CL",
    ],
    
    itp_input = "top_for_CGSB.top",
    
    solute_input = "Ligand_1.pdb params:imp mol:LIG1_name:1:LIG1_charge",
    
    out_sys = test_outdir + "/" + "_".join(["output", sysname]),
    out_top = test_outdir + "/" + "_".join(["topol", sysname]) + ".top",
    out_log = test_outdir + "/" + "_".join(["log", sysname]) + ".log",
    
    backup = False,
)




Loading topology file(s)
     Finished loading topologies. Number of moleculetypes found: 351
------------------------------ PREPROCESSING
Preprocessing lipid definitions
     Number of lipids preprocessed: 107 

Preprocessing solvent definitions
     Number of solvents preprocessed: 23 

Preprocessing ion definitions
     Number of positive ions preprocessed: 4
     Number of negative ions preprocessed: 10

Preprocessing leaflet requests
     Starting leaflet command: 1
     Number of membranes preprocessed: 1

Preprocessing solvent requests
     Starting Solvent command: 1
     Starting Solvent command: 2
     Number of solvent commands preprocessed: 2
------------------------------ PREPROCESSING COMPLETE 

------------------------------ CREATING LEAFLET BOUNDARY BOXES
Making boundary boxes for membrane nr 1
     Starting upper leaflet
          Base parameters: x=10.0nm y=10.0nm APL=0.6nm^2

     Starting lower leaflet
          Base parameters: x=10.0nm y=10.0nm APL=0.6nm^2
------